In [2]:
import pandas as pd
import numpy as np
from utils import filter_hp_uk_data, download_osm_boundaries, precise_boundary_filter

In [3]:
df=pd.read_csv('rainfall_data_transformed_final.csv',encoding='utf-8')

In [4]:
df.head()

,latitude,longitude,2010-06-01 00:00:00,2010-06-01 01:00:00,2010-06-01 02:00:00,2010-06-01 03:00:00,2010-06-01 04:00:00,2010-06-01 05:00:00,2010-06-01 06:00:00,2010-06-01 07:00:00,...,2024-09-30 14:00:00,2024-09-30 15:00:00,2024-09-30 16:00:00,2024-09-30 17:00:00,2024-09-30 18:00:00,2024-09-30 19:00:00,2024-09-30 20:00:00,2024-09-30 21:00:00,2024-09-30 22:00:00,2024-09-30 23:00:00
0,28.44,75.47,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,28.44,75.72,0.0,0.0,0.0,0.0,0.0,0.0,7.823110e-07,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,28.44,75.97,0.0,0.0,0.0,0.0,0.0,0.0,2.175569e-06,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28.44,76.22,0.0,0.0,0.0,0.0,0.0,0.0,7.420778e-06,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,28.44,76.47,0.0,0.0,0.0,0.0,0.0,0.0,5.416572e-06,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_long = df.melt(id_vars=['latitude','longitude'], var_name='datetime', value_name='rainfall')
df_long['datetime'] = pd.to_datetime(df_long['datetime'])

In [6]:
df_long

,latitude,longitude,datetime,rainfall
0,28.44,75.47,2010-06-01 00:00:00,0.0
1,28.44,75.72,2010-06-01 00:00:00,0.0
2,28.44,75.97,2010-06-01 00:00:00,0.0
3,28.44,76.22,2010-06-01 00:00:00,0.0
4,28.44,76.47,2010-06-01 00:00:00,0.0
...,...,...,...,...
19193035,32.94,79.97,2024-09-30 23:00:00,0.0
19193036,32.94,80.22,2024-09-30 23:00:00,0.0
19193037,32.94,80.47,2024-09-30 23:00:00,0.0
19193038,32.94,80.72,2024-09-30 23:00:00,0.0


In [7]:
df_long['rainfall']=df_long['rainfall']*1000  # Convert from meters to mm

In [8]:
df_long['month'] = df_long['datetime'].dt.month
df_long['year']  = df_long['datetime'].dt.year
df_mon = df_long[df_long['month'].isin([6, 7, 8, 9])]

In [10]:
df_filtered=precise_boundary_filter(df_mon, 'osm')

Using osm method for boundary filtering...
Original data points: 19193040
Downloaded OSM boundaries
Extracted OSM Boundaries


KeyError: Index(['id'], dtype='object')

In [33]:
len(df_filtered['latitude'].unique())

17

In [25]:
df_original=df_filtered.copy()

In [28]:
df_original.head()

,latitude,longitude,datetime,rainfall,month,year,id,index_right,state
192,30.44,77.47,2010-06-01,0.0,6,2010,192,0,Himachal Pradesh
214,30.69,77.22,2010-06-01,0.0,6,2010,214,0,Himachal Pradesh
215,30.69,77.47,2010-06-01,0.0,6,2010,215,0,Himachal Pradesh
216,30.69,77.72,2010-06-01,0.0,6,2010,216,0,Himachal Pradesh
236,30.94,76.97,2010-06-01,0.0,6,2010,236,0,Himachal Pradesh


In [ ]:
df_monthly = (df_mon
    .groupby(['latitude','longitude','year','month'], as_index=False)
    ['rainfall'].sum().rename(columns={'rainfall': 'monthly_rainfall'}))

In [22]:
df_monthly

,latitude,longitude,year,month,monthly_rainfall
0,28.44,75.47,2010,6,43.736321
1,28.44,75.47,2010,7,176.968401
2,28.44,75.47,2010,8,196.906941
3,28.44,75.47,2010,9,155.821892
4,28.44,75.47,2011,6,44.747408
...,...,...,...,...,...
26215,32.94,80.97,2023,9,14.335160
26216,32.94,80.97,2024,6,15.646587
26217,32.94,80.97,2024,7,12.984814
26218,32.94,80.97,2024,8,124.770023
